
### Asserts

Asserts can be globally disabled with an interpreter settings
They shouldn't be used for security checks etc.

---

### Lists

Changes look better in commits with lists in separate lines.
Also put comma behind last item to make it easier to add new items.


This is better

In [ ]:
import random
names = [
    'bob',
    'john',
    'robert',
]

Than this

In [ ]:
names = ['bob','john','robert']

And this

In [ ]:
names = [
    'bob',
    'john',
    'robert'
]

--- 
### Context managers

Context managers are classes with `__enter__` and `__exit__`.

In [27]:

class ContextManager:
    
    def __enter__(self):
        print('enter')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit')
     

In [28]:
with ContextManager():
    print('hello')

enter
hello
exit


In [29]:
with ContextManager():
    raise ValueError

enter
exit


ValueError: 

--- 

### Underscores

### `_var` - Single leading underscore  
Private variable. This is just a convention (in PEP8)  
Python will not import import modules with leading underscores automatically.
Unless defined in `__all__`

### `var_` - Single trailing underscore  
If is variable name is already taken, use this.
`set_`,`class_`,`dict_` etc.
This is just a convention.

### `__var` - Double leading underscore  
Used for name mangling.  
Interpreter changes this name to something else, to prevent collision when extended.
 

In [1]:
class TestA:
    def __init__(self):
        self.a = 1
        self._a = 1
        self.__a = 1
        
    def do_something(self):
        print(self.a)
        print(self._a)
        print(self.__a)
        
tsta = TestA()
print(dir(tsta))
tsta.do_something()

# print(tsta.a)
# print(tsta._a)
# print(tsta.__a)
# This explodes -> __a is not here

class TestB(TestA):
    def __init__(self):
        
        super().__init__()
        
        self.a = 3
        self._a = 3
        self.__a = 3

tstb = TestB()
print(dir(tstb))
tstb.do_something()


['_TestA__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
1
1
1
['_TestA__a', '_TestB__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
3
3
1



### `__var__` - Double leading and training underscores - 'Magic' methods 

### `_` - Single underscore - Unused variable

--- 

### String formatting

#### Old string formatting

Isn't deprecated, but should be avoided


In [2]:

name = 'John'
print('Hello %s' % name)


Hello John



#### New string formatting

Better than 'Old string'


In [1]:

name = 'John'
print('Hello {}'.format(name))


Hello John



#### Literal string interpolation

This should be used.
It is usually faster than just `'Hello '+ name`


In [2]:

name = 'John'
print(f'Hello {name}')


Hello John



#### Template strings

Useful when handling 'untrusted' input from users


In [3]:
from string import Template

name = 'John'

tem = Template('Hello $name')
tem.substitute(name=name)


'Hello John'


How untrusted string from user can expose secrets 


In [4]:

class Car:
    
    car_secret_key = 'SECRET'
    
    def __init__(self):
        pass

blue_car = Car()

string_from_evil_user = '{car.car_secret_key}'

print(string_from_evil_user.format(car=blue_car))


SECRET



--- 

### Functions

Functions that can accept other functions are called higher-order functions


In [6]:

def get_car_func(color):
    
    def red(brand):
        return f'Red {brand}'
    
    def blue(brand):
        return f'Blue {brand}'
    
    if color == 'red':
        return red
    else:
        return blue
    
print(get_car_func('red'))

car_fce = get_car_func('red')

print(car_fce('BMW'))


<function get_car_func.<locals>.red at 0x1073dfb70>
Red BMW



Returned function can still access the upper scope.
This is called 'Lexical closure'


In [11]:

def get_car_func(color):
    
    def red(brand):
        return f'RED {color.capitalize()} {brand}'
    
    def other(brand):
        return f'OTHER {color.capitalize()} {brand}'
    
    if color == 'red':
        return red
    else:
        return other


a = get_car_func('red')
print(a('BMW'))

b = get_car_func('blue')
print(b('BMW'))



RED Red BMW
OTHER Blue BMW



### Decorators

Simplest decorator :


In [3]:

def null_decorator(func):
    return func

def say_name():
    return 'John'

name = null_decorator(say_name)

name()


'John'


Method can be permanently decorated with `@`


In [2]:

def null_decorator(func):
    return func

@null_decorator
def say_name():
    return 'John'

say_name()


'John'


More complicated decorator


In [4]:

def title_decorator(func):
    
    def wrapper():
        original_result = func()
        new_result = original_result.title()
        
        return new_result
    
    return wrapper

@title_decorator
def say_name():
    return 'little john'

say_name()

'Little John'


Decorators are applied from bottom to top


In [5]:

def aa_decorator(func):
    
    def wrapper():
        return f'aa_{func()}_aa'
    
    return wrapper


def bb_decorator(func):
    
    def wrapper():
        return f'bb_{func()}_bb'
    
    return wrapper

@bb_decorator
@aa_decorator
def say_name():
    return 'little john'

say_name()


'bb_aa_little john_aa_bb'


This also means that deep levels of decorator stacking weill eventually have an effect on performance

#### kwargs + args


In [8]:

def aa_decorator(func):
    
    def wrapper(*args, **kwargs):
        
        print(f'{func.__name__} called with {args} and {kwargs}')
        
        return f'aa_{func()}_aa'
    
    return wrapper

# This can be useful for debugging
@aa_decorator
def say_name():
    return 'little john'

say_name()
say_name('hello')
say_name('hello', name='bean')


say_name called with () and {}
say_name called with ('hello',) and {}
say_name called with ('hello',) and {'name': 'bean'}


'aa_little john_aa'


Decorating function hides some data of the original function


In [12]:

def hello():
    """This just says hello"""
    
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

decorated_hello = aa_decorator(hello)

print(decorated_hello.__name__)
print(decorated_hello.__doc__)


hello
This just says hello
wrapper
None



Which can be fixed with `functools.wraps`.
You should use that :)


In [20]:

import functools

def aa_decorator(func):
    @functools.wraps(func)    
    def wrapper(*args, **kwargs):
        
        print(f'{func.__name__} called with {args} and {kwargs}')
        
        return f'aa_{func()}_aa'
    
    return wrapper

@aa_decorator
def hello():
    """This just says hello"""
    
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

hello
This just says hello



### *args and **kwargs

This allows functions to accept optional arguments


In [5]:

def some_func(required_param, *args, **kwargs):
    
    print(required_param)
    
    if args:
        print(f'args {args}')
        
    if kwargs:
        print(f'kwargs {kwargs}')


some_func('required')
some_func('required',1,2)
some_func('required',1,2,a='aaa', b='bb')


required
required
args (1, 2)
required
args (1, 2)
kwargs {'a': 'aaa', 'b': 'bb'}



This can be used to pass parameters from one function to another


In [5]:

def some_other_func(required_param, *args, **kwargs):
    
    print(required_param)
    print(args)
    print(kwargs)

def some_func(required_param, *args, **kwargs):
    
    print(required_param)
    print(args)
    print(kwargs)
    
    kwargs['something'] = 'else'
    args += ('aaa', )
    
    some_other_func(required_param, *args, **kwargs)
    
    
some_func('required',1,2,a='aaa', b='bb')


required
(1, 2)
{'a': 'aaa', 'b': 'bb'}
required
(1, 2, 'aaa')
{'a': 'aaa', 'b': 'bb', 'something': 'else'}
red



`Apple` passes all parameters to its parent. 
Therefore if `Fruit` significantly changes, `Apple` will be probably still working.


In [6]:
class Fruit:
    def __init__(self, color, size):
        self.color = color
        self.size = size
        
class Apple(Fruit):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.color = 'red'

apple = Apple('blue', 100)
print(apple.color)




red



### Function argument unpacking


In [11]:
def print_something(a, b, c):
    print(f'a:{a} b:{b} c:{c}')
    
print_something(0,1,2)

tuple_vec = (0,1,2)
list_vec = [0,1,2]

print_something(*tuple_vec)
print_something(*list_vec)
    
generator = [x * 2 for x in range(3)]
print(generator)

print_something(*generator)

dict_vector = {'a': -1, 'b': -2, 'c': -3}
print_something(**dict_vector)
print_something(*dict_vector)
print_something(*dict_vector.keys())
print_something(*dict_vector.values())


a:0 b:1 c:2
a:0 b:1 c:2
a:0 b:1 c:2
[0, 2, 4]
a:0 b:2 c:4
a:-1 b:-2 c:-3
a:a b:b c:c
a:a b:b c:c
a:-1 b:-2 c:-3



### "is" vs "=="


In [8]:

a = [1, 2, 3]
b = a

# Lists have same content
print(a == b)

# Lists are same instances
print(a is b)

c = list(a)
# Lists are same
print(a == c)

# But are different instances
print(a is c)


True
True
True
False



### Every class needs `__repr__`


In [17]:

class Apple:
    def __init__(self, color, size):
        self.color = color
        self.size = size
        
red_apple = Apple('red', 10)
print(red_apple)
# This is horrible

# If __str__ is missing python falls back to __repr__

class Apple:
    def __init__(self, color, size):
        self.color = color
        self.size = size
        
    def __str__(self):
        # This is used for conversion to string
        return f'Class Apple color:{self.color} size:{self.size}cm'

    def __repr__(self):
        # This is used when debugging to terminal
        # Should be 'more technical'
        # !r calls repr() in object
        return f'{self.__class__.__name__}({self.color!r} {self.size!r})'
        
        
red_apple = Apple('red', 10)
print(red_apple)
red_apple
print(str(red_apple))
print(repr(red_apple))



Class Apple color:red size:10cm
Class Apple color:red size:10cm
Apple('red' 10)


### Defining your own exception classes


In [18]:
# This is considered a bad idea, because exception doesn't describe what is wrong
def connection(address):
    if not isinstance(address, str):
        raise ValueError


connection(10)


ValueError: 

In [ ]:
# This is better

class BaseConnectionError(ValueError):
    pass

class ConnectionAddressNotString(BaseConnectionError):
    pass

class ConnectionAddressNotIPv6(BaseConnectionError):
    pass

def connection(address):
    if not isinstance(address, str):
        raise ConnectionAddressNotString

try:
    connection('127.0.0.1')
except BaseConnectionError:
    pass


### Cloning objects
* Shallow copy - Is only one level deep
* Deep copy - Copies recursively

#### Shallow copy

In [22]:
a = [[1,2,3],[4,5,6],[7,8,9]]
b = list(a)
# This makes only shallow copy
# Same as copy.copy(a)

print(a)
print(b)

a.append([5555])
print()

print(a)
print(b)

a[0].append(12345)
print()

# Both a and b are updated
print(a)
print(b)


[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [5555]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

[[1, 2, 3, 12345], [4, 5, 6], [7, 8, 9], [5555]]
[[1, 2, 3, 12345], [4, 5, 6], [7, 8, 9]]


#### Deep copy

In [23]:
import copy

a = [[1,2,3],[4,5,6],[7,8,9]]
b = copy.deepcopy(a)

print(a)
print(b)

a.append([5555])
print()

print(a)
print(b)

a[0].append(12345)
print()

# Both a and b are updated
print(a)
print(b)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [5555]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

[[1, 2, 3, 12345], [4, 5, 6], [7, 8, 9], [5555]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


#### With classes

In [29]:

class Book:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return f'Book({self.name!r})'

class Library:
    def __init__(self, books):
        self.books = books
    
    def __repr__(self):
        return f'Library({self.books!r})'


ba = Book('a')
bb = Book('b')
bc = Book('c')

lib = Library([ba, bb, bc])
print(lib)

lib_b = copy.copy(lib)
print(lib_b)

print()

ba.name = 'boook'

print(lib)
print(lib_b)
print()

# Now deep copy

ba = Book('a')
bb = Book('b')
bc = Book('c')

lib = Library([ba, bb, bc])
print(lib)

lib_b = copy.deepcopy(lib)
print(lib_b)

print()

ba.name = 'boook'

print(lib)
print(lib_b)



Library([Book('a'), Book('b'), Book('c')])
Library([Book('a'), Book('b'), Book('c')])

Library([Book('boook'), Book('b'), Book('c')])
Library([Book('boook'), Book('b'), Book('c')])

Library([Book('a'), Book('b'), Book('c')])
Library([Book('a'), Book('b'), Book('c')])

Library([Book('boook'), Book('b'), Book('c')])
Library([Book('a'), Book('b'), Book('c')])


### Abstract Base Classes

This works but have some downsides

In [32]:
class BaseClass:
    def a(self):
        raise NotImplementedError()
    
    def b(self):
        raise NotImplementedError()
    
class OtherClass(BaseClass):
    def a(self):
        print('hello')


base = BaseClass()
# This is wrong

other = OtherClass()
# We still don't know if all methods are implemented here
other.b()



NotImplementedError: 

With use of ABCs


In [34]:

from abc import ABCMeta, abstractmethod

class BaseClass(metaclass=ABCMeta):
    @abstractmethod
    def a(self):
        pass
    
    @abstractmethod
    def b(self):
        pass
    
class OtherClass(BaseClass):
    def a(self):
        print('Hello')


base = BaseClass()
# This explodes

other = OtherClass()
other.a()
# This explodes too


TypeError: Can't instantiate abstract class BaseClass with abstract methods a, b


### Named tuples


In [48]:
from collections import namedtuple

# This is same thing
Apple = namedtuple('Apple', ['color', 'size'])
Apple = namedtuple('Apple', 'color size')


red_apple = Apple('red', 30)
print(red_apple)
print(red_apple.color)
print(red_apple.size)

print(red_apple[0])
print(red_apple[1])

color, size = red_apple
print(color)
print(size)

print(*red_apple)

# red_apple.color = 'blue'
# This explodes

# You can even subclass Named tuples
class FlyingApple(Apple):
    def fly(self):
        print(f'Apple {self.color} flies')
        
red_color_fly = FlyingApple('red', 30)
print(red_color_fly)
red_color_fly.fly()

print(red_color_fly._fields)
print(red_color_fly._asdict())

# This creates new instance
new_apple = red_color_fly._replace(color='blue')
print(new_apple)

# New instance from iterable
new_apple = FlyingApple._make(['blue', 40])


Apple(color='red', size=30)
red
30
red
30
red
30
red 30
FlyingApple(color='red', size=30)
Apple red flies
('color', 'size')
OrderedDict([('color', 'red'), ('size', 30)])
FlyingApple(color='blue', size=30)



### Class and Instance variables

*Class variables* are declared inside the class definition, but outside any instance methods
*Instance variables* are always tied to a particular instance


In [51]:

class Car:
    num_wheels = 4
    # Class variable
    
    def __init__(self, color):
        self.color = color
        # Instance variable
        
car_a = Car('red')
car_b = Car('blue')

print(car_a.num_wheels)
print(car_b.num_wheels)

Car.num_wheels = 10

print(car_a.num_wheels)
print(car_b.num_wheels)

# This creates instance variable which shadows class variable
# This is slightly horrible
car_a.num_wheels = 6
print(car_a.num_wheels)
print(car_a.__class__.num_wheels)


4
4
10
10
6
10



### Instance, Class and Static methods


In [56]:

class SomeClass:
    
    def instanceMethod(self):
        # This is normal method
        # You can reach instance through self
        # And also class through self.__class__
        pass
    
    @classmethod
    def classMethod(cls):
        # This can access only class, not instance
        pass
    
    @staticmethod
    def staticMethod():
        # This can't reach class
        # Static methods are primarily used for namespacing
        # This can be reached as SomeClass.staticMethod()
        # or instance_class.staticMethod()
        pass
    
class Car:
    
    def __init__(self, color, max_speed, brand):
        self.color = color
        self.max_speed = max_speed
        self.brand = brand
        
    def __repr__(self):
        return f'Car({self.color!r}, {self.max_speed!r}, {self.brand!r})'
    
    @classmethod
    def fast_car(cls):
        return cls('blue', 300, 'BMW')
    
    @classmethod
    def big_car(cls):
        return cls('red', 100, 'Tatra')
    
    @staticmethod
    def max_weight(speed):
        return speed / 4
    
print(Car.fast_car())
print(Car.big_car())

print(Car.max_weight(100))

Car('blue', 300, 'BMW')
Car('red', 100, 'Tatra')
25.0



### Common data structures in Python

#### Dict

Are indexed by keys, that must be hashable `__hash__` type and can be copared `__eq__`  
Tuples can be used as keys if all elements in tuple are hashable.  
Dicts are based on hash table and have O(1) complexity for lookup, insert and delete.

Only from Python 3.7.4 dicts are ordered by default.
If dict order is needed, use `collections.OrderedDict`



In [61]:
from collections import ChainMap

# ChainMap groups multiple dicts into one

dict_a = {'a': 1, 'b': 2}
dict_b = {'c': 3, 'd': 4}
dict_c = {'f': 6, 'a': 100}

chain = ChainMap(dict_a, dict_b, dict_c)

print(chain['a'])
print(chain['f'])
print(chain['aaaa'])


1
6


KeyError: 'aaaa'

In [63]:

from types import MappingProxyType
# MappingProxyType wraps dict and makes it read only

read_only_a = MappingProxyType(dict_a)
print(read_only_a['a'])
read_only_a['a'] = 33


TypeError: 'mappingproxy' object does not support item assignment

#### `list` - Mutable Dynamic Array
Are implemented as dynamic lists
#### `tuple` - Immutable Containers
Elements can't be added or removed dynamically.  
Everything must be defined at creation time
#### `array.array` - Basic typed arrays
Space efficient storage of C-style data types.  
Arrays are mutable and behaves similarly to lists, except they are typed to same data type.


In [67]:
import array

arr_f = array.array('f', [1.0, 2.2, 3.3])
print(arr_f)
arr_f[0] = 10
print(arr_f)
arr_f.append(55)
print(arr_f)


array('f', [1.0, 2.200000047683716, 3.299999952316284])
array('f', [10.0, 2.200000047683716, 3.299999952316284])
array('f', [10.0, 2.200000047683716, 3.299999952316284, 55.0])


#### `str` - Immutable Arrays of Unicode characters
Each element is a single unicode character.  
str is immutable.


In [70]:
arr = 'abcdef'
print(arr)
print(arr[1])
arr[1] = 'g'


abcdef
b


TypeError: 'str' object does not support item assignment

#### `bytes` - Immutable arrays of single bytes
Can store ints `0 <= x <= 255`. Bytes are space efficient.


In [73]:
arr = bytes((0, 1, 2, 3, 4))
print(arr)
print(arr[2])

arr = bytes((1,2,3,444))


b'\x00\x01\x02\x03\x04'
2


ValueError: bytes must be in range(0, 256)

#### `bytearray` - Mutable Arrays of Single bytes
Very similar to bytes, but can be modified

In [80]:

arr = bytearray((1,2,3,4))
print(arr)

arr[1] = 4
print(arr)

arr.append(10)
print(arr)

# Can be converted to bytes
print(bytes(arr))


bytearray(b'\x01\x02\x03\x04')
bytearray(b'\x01\x04\x03\x04')
bytearray(b'\x01\x04\x03\x04\n')
b'\x01\x04\x03\x04\n'


#### `Struct` - Converts data between C structs and Python

In [83]:
from struct import Struct

My_struct = Struct('i?f')

data = My_struct.pack(100, True, 111.1)

print(data)

print(My_struct.unpack(data))


b'd\x00\x00\x00\x01\x00\x00\x0033\xdeB'
(100, True, 111.0999984741211)


### Sets and Multisets
Set is and unordered collection of object that not allow duplicate element.  

#### Standard set

In [ ]:
chars = {'a', 'b', 'c', 'd'}
print(chars)
print('a' in chars)

another_chars = set('mnbvvca')
print(another_chars)
print(another_chars.intersection(chars))

chars.add('r')


#### Frozen Set
Immutable set

In [90]:

chars_frozen = frozenset({'a', 'a', 'b', 'c'})
print(chars_frozen)
chars_frozen.add('a')

frozenset({'c', 'b', 'a'})


AttributeError: 'frozenset' object has no attribute 'add'

#### `collections.Counter` - Multiset

In [95]:

from collections import Counter

tools = Counter()
tools.update({'a': 1, 'b': 3})
print(tools)
tools.update({'a': 6, 'b': 3})
print(tools)

print(len(tools))
# Number of unique elements

print(sum(tools.values()))
# Sum of all values


Counter({'b': 3, 'a': 1})
Counter({'a': 7, 'b': 6})
2
13


### Stacks (LIFO)

#### List
Simple built in stack
Is implemented as dynamic array which over allocates space. Therefore offers O(1) most of the time.
Performance can be inconsistent.
Offers fast O(1) access time.
To get O(1) data must be added to the end (`append()`) and removed also from end (`pop()`).
Adding and removing from front can be O(n)


In [98]:
stack = []
stack.append('a')
stack.append('b')
stack.append('c')

print(stack)

print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())


['a', 'b', 'c']
c
b
a


IndexError: pop from empty list

#### `collections.deque` - Fast and robust stacks
Double-ended queue that supports adding and removing elements with O(1)


In [101]:
from collections import deque

stack = deque()
stack.append('a')
stack.append('b')
stack.append('c')

print(stack)

print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())


deque(['a', 'b', 'c'])
c
b
a


IndexError: pop from an empty deque

#### `queue.LifoQueue` - Synchronized queue for parallel computing
Supports multiple concurrent consumers and producers


In [106]:
from queue import LifoQueue

stack = LifoQueue()
stack.put('a')
stack.put('b')
stack.put('c')

print(stack)

print(stack.get())
print(stack.get())
print(stack.get())
print(stack.get_nowait())


c
b
a


Empty: 


Summary :
* `list` - Backed by dynamic array - great for random access, but sometimes reallocates. You shold access only items on end for O(1)
* `deque` - Double linked list - Consistent O(1)

### Queues (FIFO)

#### `list` - Really slow fifo
Dynamically allocated array - shoud be avoided for FIFO


In [108]:

q = []
q.append('a')
q.append('b')
q.append('c')

print(q)

print(q.pop(0))
print(q.pop(0))
print(q.pop(0))


['a', 'b', 'c']
a
b
c


#### `collections.deque` - Can be used as queue
Double linked list. O(1) for accessing ends O(n) for random access in array


In [110]:
from collections import deque

q = deque()
q.append('a')
q.append('b')
q.append('c')

print(q)

print(q.popleft())
print(q.popleft())
print(q.popleft())
print(q.popleft())


deque(['a', 'b', 'c'])
a
b
c


IndexError: pop from an empty deque

#### `queue.Queue` - Synchronized for parallel computing

In [113]:
from queue import Queue

q = Queue()
q.put('a')
q.put('b')
q.put('c')

print(q)

print(q.get())
print(q.get())
print(q.get())


a
b
c


#### `mutliprocessing.Queue` - Shared job queue
Special queue for multiprocessing.  
Can transfer anything Picklable.  

In [115]:
from multiprocessing import Queue

q = Queue()
q.put('a')
q.put('b')
q.put('c')

print(q)

print(q.get())
print(q.get())
print(q.get())


a
b
c


### Priority queues
Something like normal Queue, but returns highest priority element, not the last one

#### `list` - Manually sorting
Slow but usable. Must be manually resorted after isertion.

In [118]:

q = []

q.append((20, 'd'))
q.append((10, 'a'))
q.append((30, 'b'))
q.append((60, 'c'))


q.sort(reverse=True)

print(q.pop())
print(q.pop())
print(q.pop())
print(q.pop())


(10, 'a')
(20, 'd')
(30, 'b')
(60, 'c')


#### `heapq` - List based binary heaps
Supports insertion and extraction at O(log n)

In [121]:

import heapq

q = []

heapq.heappush(q, (80, 'd'))
heapq.heappush(q, (20, 'a'))
heapq.heappush(q, (30, 'b'))
heapq.heappush(q, (60, 'c'))

print(q)

print(heapq.heappop(q))
print(heapq.heappop(q))
print(heapq.heappop(q))
print(heapq.heappop(q))


[(20, 'a'), (60, 'c'), (30, 'b'), (80, 'd')]
(20, 'a')
(30, 'b')
(60, 'c')
(80, 'd')


#### `queue.PriorityQueue` - Beutiful Priority queue
Uses `heapq` internally but supports multiple producers and consumers.  
Offers class based interface, opposed to `heapq`


In [123]:
from queue import PriorityQueue

q = PriorityQueue()
q.put((30, 'a'))
q.put((20, 'b'))
q.put((60, 'c'))
q.put((40, 'd'))

print(q)

print(q.get())
print(q.get())
print(q.get())
print(q.get())


(20, 'b')
(30, 'a')
(40, 'd')
(60, 'c')


### Comprehensions


In [22]:
# List Comprehensions
a = [x * 2 for x in range(10)]
print(a)

a = [i * 2 for i in range(10) if i % 2 == 0]
print(a)

a = [i if i % 2 == 0 else 'a' for i in range(10)]
print(a)

a = [i if i == 0 else 'b' for i in range(10) if i % 2 == 0]
print(a)

a = [i if i[1] == 'b' else None for i in ['aa', 'ab', 'ac', 'bb'] if i.startswith('a')]
print(a)

str_list = ['inv-0', 'inv-1', 'inv-2', 'val-0', 'val-1', 'val-2']
a = [i if int(i[-1]) == 0 else 'non-zero' for i in str_list if i.startswith('val')]
print(a)

# Sets
a = {i for i in range(10)}
print(a)

a = {i for i in [0, 0, 1, 1]}
print(a)

# Dicts

names = ['a', 'b', 'c', 'd']
nums = [0, 1, 2, 3]

a = {i[0]: i[1] for i in zip(names, nums)}
print(a)

a = {i[0]: i[1] for i in zip(names, nums) if i[1] % 2 == 0}
print(a)


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
[0, 4, 8, 12, 16]
[0, 'a', 2, 'a', 4, 'a', 6, 'a', 8, 'a']
[0, 'b', 'b', 'b', 'b']
[None, 'ab', None]
['val-0', 'non-zero', 'non-zero']
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{0, 1}
{'a': 0, 'b': 1, 'c': 2, 'd': 3}
{'a': 0, 'c': 2}


### List Slicing

In [41]:

a = [0, 1, 2, 3, 4, 5, 6]
print(a[1:4:2])
print(a[1:4])
print(a[::2])
print(a[::-1])
print(list(reversed(a)))

print(id(a))
# Deletes all elements from the list. Object address is still same
print(a[:])
del a[:]
print(a[:])
print(id(a))
a = []
# New object, destroys previous references
print(id(a))
a.clear()
# Same thing for Python3
print(id(a))


[1, 3]
[1, 2, 3]
[0, 2, 4, 6]
[6, 5, 4, 3, 2, 1, 0]
[6, 5, 4, 3, 2, 1, 0]
4571729352
[0, 1, 2, 3, 4, 5, 6]
[]
4571729352
4568597320
4568597320


In [47]:
c = [0, 1, 2, 3, 4]
d = c

print(id(c))
print(id(d))

print(c is d)

c[:] = ['a', 'b', 'c']

print(c)
print(d)

print(id(c))
print(id(d))

print(c is d)

f = c[:]
print(f)

print(id(c))
print(id(d))
print(id(f))


4568496712
4568496712
True
['a', 'b', 'c']
['a', 'b', 'c']
4568496712
4568496712
True
['a', 'b', 'c']
4568496712
4568496712
4571780744
